In [1]:
import pandas as pd
import glob # to scan files from a folder and get it in a list of names with locations
from functools import reduce #to be able to merge the items within a list

### THe below steps will start post conversion of the quarterly earnings PDF into excel for the 'mobility earnings' table. use the quarterly tables in the excel format in this code to get them converted for modelling

In [2]:
path = r'C:\\Users\\z0047cfr\\Project Work\\Factory Demand Model\\Input Data\\Mobility_Earnings' # use your path
csv_files = glob.glob(path + "/*.csv") # using glob function to extract list of quarterly files

In [6]:
#full utilization rates


li = []

for filename in csv_files:
    F2_internal0 = pd.read_csv(filename)
    #F2_internal = pd.read_csv(filename, dtype = {'(in millions of €)': float}, thousands = ',')
    F2_internal0.columns= ['Earnings_field', 'Earnings_value']
    # renaming variables as required
    
    F2_internal0["Earnings_field"]= F2_internal0["Earnings_field"].str.replace("Adjusted EBITA", "Profit", case = False) 
    F2_internal0["Earnings_field"]= F2_internal0["Earnings_field"].str.replace("excl. severance", "Profit excl. severance", case = False)
    F2_internal0["Earnings_value"]= F2_internal0["Earnings_value"].str.replace(",", "", case = False) 
    F2_internal0["Earnings_value"]= F2_internal0["Earnings_value"].str.replace("%", "", case = False)
    #F2_internal["Earnings_value"]= F2_internal["Earnings_value"].str.replace("(", "-", case = False)
    
    # since the data for each quarter is in a seperate file, the file name is used to identify quarter and input it in varname
    year = filename[87:91]
    qtr = filename[92:94]
    qtr_yr = qtr + '_' + year 
    names = F2_internal0.columns.tolist()
    names[names.index('Earnings_value')] = qtr_yr
    F2_internal0.columns = names
        
    li.append(F2_internal0)
#after seperate datasets are created for eachquarterly file, the below step will merge it together on the time period
F2_internal_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Earnings_field'],
                                            how='outer'), li)
    

#F2_3.head()
#F2_3.columns
#li[1]
#frame
F2_internal_merged.head(10)

,Earnings_field,Q4_2014,Q1_2015,Q2_2015,Q3_2015,Q4_2015,Q1_2016,Q2_2016,Q3_2016,Q4_2016,...,Q2_2018,Q3_2018,Q4_2018,Q1_2019,Q2_2019,Q3_2019,q4_2019,Q1_2020,Q2_2020,Q3_2020
0,Orders,1271,1271,3782,2821,2387,2663,1825,1112,2274,...,2416,2505,2820,4546,3524,2996,1827,1665,2384,3040
1,Revenue,1858,1858,1836,1817,1998,2044,1915,1795,2070,...,2100,2131,2348,2174,2116,2126,2500,2180,2263,2162
2,Profit,155,155,157,105,171,193,153,158,173,...,233,177,236,236,228,220,299,219,210,153
3,therein: severance,-1,(1),(3),(30),(34),(3),(3),(4),(5),...,(4),−,(4),(2),(7),(3),(7),-6,-4,-5
4,Profit margin,8.40,8.4,8.6,5.8,8.6,9.4,8.0,8.8,8.4,...,11.1,8.3,10.1,10.9,10.8,10.4,12.0,10.00,9.30,7.10
5,Profit excl. severance,8.40,8.4,8.7,7.4,10.2,9.6,8.2,9.1,8.6,...,11.3,8.3,10.2,11.0,11.1,10.5,12.3,10.30,9.50,7.30


In [9]:
# transpose data to bring to desired format
F2_internal_1 = F2_internal_merged.T
F2_internal_1.head()
F2_internal_2 =F2_internal_1[1:]
# renaming column names to include all inportant information like data being in millions
F2_internal_2.columns = ['Orders_millions_Q', 'Revenue_millions_Q', 'Profit_Q', 'Severance_Amount_millions_Q', 'Profit_margin_perc_Q', 'Profit_post_severance_perc_Q']
F2_internal_2.Severance_Amount_millions_Q = F2_internal_2.Severance_Amount_millions_Q.astype(str).str.replace('\((.*)\)', '\\1').astype(str).str.replace('-', '').astype(str).str.replace('−', '')
F2_internal_2.loc[:, ('Severance_Amount_millions_Q')] = F2_internal_2.loc[:, ('Severance_Amount_millions_Q')].astype(str).str.replace('\((.*)\)', '\\1').astype(str).str.replace('-', '').astype(str).str.replace('−', '')
F2_internal = F2_internal_2.apply(pd.to_numeric)

F2_internal.head()

,Orders_millions_Q,Revenue_millions_Q,Profit_Q,Severance_Amount_millions_Q,Profit_margin_perc_Q,Profit_post_severance_perc_Q
Q4_2014,1271,1858,155,1.0,8.4,8.4
Q1_2015,1271,1858,155,1.0,8.4,8.4
Q2_2015,3782,1836,157,3.0,8.6,8.7
Q3_2015,2821,1817,105,30.0,5.8,7.4
Q4_2015,2387,1998,171,34.0,8.6,10.2


In [10]:
# convert period to index
F2_internal['period'] = F2_internal.index
F2_internal.head()

,Orders_millions_Q,Revenue_millions_Q,Profit_Q,Severance_Amount_millions_Q,Profit_margin_perc_Q,Profit_post_severance_perc_Q,period
Q4_2014,1271,1858,155,1.0,8.4,8.4,Q4_2014
Q1_2015,1271,1858,155,1.0,8.4,8.4,Q1_2015
Q2_2015,3782,1836,157,3.0,8.6,8.7,Q2_2015
Q3_2015,2821,1817,105,30.0,5.8,7.4,Q3_2015
Q4_2015,2387,1998,171,34.0,8.6,10.2,Q4_2015


In [11]:
# date time conversion
F2_internal['DATE'] = F2_internal.period.str.replace('_',' ').str.replace('q1','03 01').str.replace('q2','06 01').str.replace('q3','09 01').str.replace('q4','12 01').str.replace('_',' ').str.replace('Q1','03 01').str.replace('Q2','06 01').str.replace('Q3','09 01').str.replace('Q4','12 01')
F2_internal['DATE'] =   pd.to_datetime(F2_internal.DATE)
F2_internal.DATE.head()

Q4_2014   2014-12-01
Q1_2015   2015-03-01
Q2_2015   2015-06-01
Q3_2015   2015-09-01
Q4_2015   2015-12-01
Name: DATE, dtype: datetime64[ns]

The data provided is as of fiscal year. we are modelling as of calendar year. 
A fiscal year starts in Oct of the pervious year. Basically three months before the calendar year. 
 we can remove three months from current data dates to convert fiscal year to calendar year.

In [12]:
F2_internal['DATE1']  = F2_internal.DATE - pd.DateOffset(months=3)
F2_internal.head()

,Orders_millions_Q,Revenue_millions_Q,Profit_Q,Severance_Amount_millions_Q,Profit_margin_perc_Q,Profit_post_severance_perc_Q,period,DATE,DATE1
Q4_2014,1271,1858,155,1.0,8.4,8.4,Q4_2014,2014-12-01,2014-09-01
Q1_2015,1271,1858,155,1.0,8.4,8.4,Q1_2015,2015-03-01,2014-12-01
Q2_2015,3782,1836,157,3.0,8.6,8.7,Q2_2015,2015-06-01,2015-03-01
Q3_2015,2821,1817,105,30.0,5.8,7.4,Q3_2015,2015-09-01,2015-06-01
Q4_2015,2387,1998,171,34.0,8.6,10.2,Q4_2015,2015-12-01,2015-09-01


In [13]:
F2_internal = F2_internal.drop( columns = ['period'])
F2_internal = F2_internal.set_index('DATE1')
F2_internal = F2_internal.sort_index()

In [14]:
# convert quarterly data to monthly by backfilling. 
F2_internal = F2_internal.resample('MS', convention = 'end').bfill()
#F2_internal.DATE.tail(5), F2_internal.DATE1.tail(5)

In [15]:
F2_internal = F2_internal.drop( columns = [ 'DATE' ])
#F2_internal.dtypes
F2_internal.head(6)

,Orders_millions_Q,Revenue_millions_Q,Profit_Q,Severance_Amount_millions_Q,Profit_margin_perc_Q,Profit_post_severance_perc_Q
DATE1,,,,,,
2014-09-01,1271,1858,155,1.0,8.4,8.4
2014-10-01,1271,1858,155,1.0,8.4,8.4
2014-11-01,1271,1858,155,1.0,8.4,8.4
2014-12-01,1271,1858,155,1.0,8.4,8.4
2015-01-01,3782,1836,157,3.0,8.6,8.7
2015-02-01,3782,1836,157,3.0,8.6,8.7


In [16]:
F2_internal = F2_internal.sort_index()
F2_internal.head()

,Orders_millions_Q,Revenue_millions_Q,Profit_Q,Severance_Amount_millions_Q,Profit_margin_perc_Q,Profit_post_severance_perc_Q
DATE1,,,,,,
2014-09-01,1271,1858,155,1.0,8.4,8.4
2014-10-01,1271,1858,155,1.0,8.4,8.4
2014-11-01,1271,1858,155,1.0,8.4,8.4
2014-12-01,1271,1858,155,1.0,8.4,8.4
2015-01-01,3782,1836,157,3.0,8.6,8.7


In [17]:
#code snippet checks
file1 = path + '\\2014-Q4-earnings-release-e.csv'
data = pd.read_csv(file1)
#file1
data.head()
#data.shape
data.columns

Index(['(in millions of €)', 'FY 2015'], dtype='object')